In [ ]:
import torch
import torch.nn as nn
from model import TinyTransformer
from dataset import get_batch_split, vocab_size, stoi, decode
import math

In [21]:
device = (
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Using device: mps


In [ ]:
model = TinyTransformer(vocab_size, n_embd=128, block_size=128, n_heads=4, n_layers=4).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
loss_fn = nn.CrossEntropyLoss()

batch_size = 64
steps = 3000
eval_interval = 500
eval_steps = 50

@torch.no_grad()
def estimate_loss():
    model.eval()
    out = {}
    for split in ['train', 'val']:
        losses = torch.zeros(eval_steps)
        for k in range(eval_steps):
            x, y = get_batch_split(split, block_size=128, batch_size=batch_size, device=device)
            logits = model(x)
            B, T, C = logits.shape
            loss = loss_fn(logits.view(B*T, C), y.view(B*T))
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

for step in range(steps):
    x, y = get_batch_split('train', block_size=128, batch_size=batch_size, device=device)

    logits = model(x)
    B, T, C = logits.shape
    loss = loss_fn(logits.view(B*T, C), y.view(B*T))

    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) 
    optimizer.step()
    
    if step % eval_interval == 0 or step == steps - 1:
        losses = estimate_loss()
        print(f"Step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

torch.save(model.state_dict(), 'tiny_shakespeare_model.pth')
print("Model saved to tiny_shakespeare_model.pth")


Step 0: train loss 3.9294, val loss 3.9417
Step 500: train loss 2.0626, val loss 2.1128
Step 1000: train loss 1.7313, val loss 1.8826
Step 1500: train loss 1.5735, val loss 1.7542
Step 1999: train loss 1.4840, val loss 1.6783


In [29]:
def generate(model, start, length=100, temperature=0.8, top_k=40):
    model.eval()
    with torch.no_grad():
        x = torch.tensor([stoi[s] for s in start], dtype=torch.long).unsqueeze(0).to(device)
        idx = model.generate(x, max_new_tokens=length, temperature=temperature, top_k=top_k)
        return decode(idx[0].tolist())

print(generate(model, "ROMEO:", length=200))

ROMEO:
What not him in undoners'd with been since,
Where stay, and repriction.

MERCUSA:
Or mother, may, and thoughts awhilest shall as them love
chind me still to many like of the lambs, sine,
And they ten
